In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 
import numpy as np
import os

In [ ]:
dossier=os.listdir("/content/drive/MyDrive/PFE_Oumaima/Adience/Adience_Data_Aug1/Femme")
Femme=np.zeros((len(dossier),64,64,3))
Labels_F=[]
for i in range(len(dossier)):
  nom_image='/content/drive/MyDrive/PFE_Oumaima/Adience/Adience_Data_Aug1/Femme/'+dossier[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Femme[i]=image
  Labels_F.append(0)

In [ ]:
dossier1=os.listdir("/content/drive/MyDrive/PFE_Oumaima/Adience/Adience_Data_Aug1/Homme")
Homme=np.zeros((len(dossier1),64,64,3))
Labels_H=[]
for i in range(len(dossier1)):
  nom_image='/content/drive/MyDrive/PFE_Oumaima/Adience/Adience_Data_Aug1/Homme/'+dossier1[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Homme[i]=image
  Labels_H.append(1)

In [ ]:
def train (x):
  return int(x*0.8)

In [ ]:
images_train_Homme=Homme[:train(len(Homme))]
images_test_Homme=Homme[train(len(Homme)):]
labels_train_Homme=Labels_H[:train(len(Labels_H))]
labels_test_Homme=Labels_H[train(len(Labels_H)):]

images_train_Femme=Femme[:train(len(Femme))]
images_test_Femme=Femme[train(len(Femme)):]
labels_train_Femme=Labels_F[:train(len(Labels_F))]
labels_test_Femme=Labels_F[train(len(Labels_F)):]

In [ ]:
X_train=np.concatenate((images_train_Homme,images_train_Femme),axis=0)
Y_train=np.concatenate((labels_train_Homme,labels_train_Femme),axis=0)
X_test=np.concatenate((images_test_Homme,images_test_Femme),axis=0)
Y_test=np.concatenate((labels_test_Homme,labels_test_Femme),axis=0)

In [ ]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

16002
16002
4002
4002


In [ ]:
# importing the libraries
from keras.models import Model
from keras.layers import Flatten, Dense
from keras.applications import vgg19
from keras.layers import Dropout,Dense,Conv2D,GlobalAveragePooling2D
#from keras.preprocessing import image

num_classes=1
IMAGE_SIZE = [64,64]  

# loading the weights of VGG16 without the top layer. These weights are trained on Imagenet dataset.
vgg19 = vgg19.VGG19(input_shape = IMAGE_SIZE + [3],weights='imagenet',include_top=False)

# this will exclude the initial layers from training phase as there are already been trained.
for layer in vgg19.layers:
    layer.trainable = False
x = Flatten()(vgg19.output)
#x2= Dense(1024, activation = 'relu')(x) # adding the output layer with softmax function as this is a multi label classification problem.
#x1= Dense(512, activation = 'relu')(x)   # we can add a new fully connected layer but it will increase the execution time.
output_layer = Dense(num_classes, activation = 'sigmoid')(x)  # adding the output layer with softmax function as this is a multi label classification problem.

model = Model(inputs = vgg19.input, outputs = output_layer)

80134624/80134624 [==============================] - 1s 0us/step


In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [ ]:
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=['accuracy'])

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)
X_test = np.array(X_test)
Y_test = np.array(Y_test)


In [ ]:
model.fit(X_train,Y_train, batch_size=100, epochs=1000, validation_data=(X_test,Y_test))

Epoch 1/1000
161/161 [==============================] - 21s 68ms/step - loss: 4.5894 - accuracy: 0.6133 - val_loss: 3.4953 - val_accuracy: 0.6484
Epoch 2/1000
161/161 [==============================] - 10s 63ms/step - loss: 2.9005 - accuracy: 0.6717 - val_loss: 2.5844 - val_accuracy: 0.6789
Epoch 3/1000
161/161 [==============================] - 9s 59ms/step - loss: 2.3290 - accuracy: 0.6941 - val_loss: 2.1048 - val_accuracy: 0.6979
Epoch 4/1000
161/161 [==============================] - 9s 59ms/step - loss: 2.0038 - accuracy: 0.7024 - val_loss: 1.8944 - val_accuracy: 0.7041
Epoch 5/1000
161/161 [==============================] - 9s 59ms/step - loss: 1.7798 - accuracy: 0.7128 - val_loss: 1.7229 - val_accuracy: 0.7134
Epoch 6/1000
161/161 [==============================] - 10s 65ms/step - loss: 1.6112 - accuracy: 0.7237 - val_loss: 1.7267 - val_accuracy: 0.7074
Epoch 7/1000
161/161 [==============================] - 10s 61ms/step - loss: 1.4705 - accuracy: 0.7318 - val_loss: 1.5521 - va

In [ ]:
model.evaluate(X_test,Y_test)

126/126 [==============================] - 3s 18ms/step - loss: 0.7884 - accuracy: 0.6977


[0.7884448766708374, 0.6976511478424072]

In [ ]:
model.save('/content/drive/MyDrive/PFE_Oumaima/Adience/VGG19_Adience_Data_Aug1_Gender_Recognition.h5')

In [ ]:
pip install keract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import keras
model=keras.models.load_model('/content/drive/MyDrive/PFE_Oumaima/Adience/VGG19_Adience_Data_Aug1_Gender_Recognition.h5')

In [ ]:
from keract import get_activations
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [ ]:
activations1 = get_activations(model,X_train[:1000],layer_names='flatten')

In [ ]:
activations2 = get_activations(model,X_train[1000:2000],layer_names='flatten')
activations3 = get_activations(model,X_train[2000:3000],layer_names='flatten')
activations4 = get_activations(model,X_train[3000:4000],layer_names='flatten')
activations5 = get_activations(model,X_train[4000:5000],layer_names='flatten')
activations6 = get_activations(model,X_train[5000:6000],layer_names='flatten')
activations7 = get_activations(model,X_train[6000:7000],layer_names='flatten')
activations8 = get_activations(model,X_train[7000:8000],layer_names='flatten')
activations9 = get_activations(model,X_train[8000:9000],layer_names='flatten')
activations10 = get_activations(model,X_train[9000:10000],layer_names='flatten')
activations11= get_activations(model,X_train[10000:11000],layer_names='flatten')
activations12= get_activations(model,X_train[11000:12000],layer_names='flatten')
activations13= get_activations(model,X_train[12000:13000],layer_names='flatten')
activations14= get_activations(model,X_train[13000:14000],layer_names='flatten')
activations15= get_activations(model,X_train[14000:15000],layer_names='flatten')
activations16= get_activations(model,X_train[15000:],layer_names='flatten')


In [ ]:
X_train1= activations1
X_train1= X_train1["flatten"]
X_train2= activations2
X_train2= X_train2["flatten"]
X_train3= activations3
X_train3= X_train3["flatten"]
X_train4= activations4
X_train4= X_train4["flatten"]
X_train5= activations5
X_train5= X_train5["flatten"]
X_train6= activations6
X_train6= X_train6["flatten"]
X_train7= activations7
X_train7= X_train7["flatten"]
X_train8= activations8
X_train8= X_train8["flatten"]
X_train9= activations9
X_train9= X_train9["flatten"]
X_train10= activations10
X_train10= X_train10["flatten"]
X_train11= activations11
X_train11= X_train11["flatten"]
X_train12= activations12
X_train12= X_train12["flatten"]
X_train13= activations13
X_train13= X_train13["flatten"]
X_train14= activations14
X_train14= X_train14["flatten"]
X_train15= activations15
X_train15= X_train15["flatten"]
X_train16= activations16
X_train16= X_train16["flatten"]


In [ ]:
X_train =np.concatenate((X_train1,X_train2,X_train3,X_train4,X_train5,X_train6,X_train7,X_train8,X_train9,X_train10,X_train11,X_train12,X_train13,X_train14,X_train15,X_train16), axis=0)


In [ ]:
activations17 = get_activations(model,X_test[:1000],layer_names='flatten')
activations18 = get_activations(model,X_test[1000:2000],layer_names='flatten')
activations19 = get_activations(model,X_test[2000:3000],layer_names='flatten')
activations20 = get_activations(model,X_test[3000:],layer_names='flatten')

In [ ]:
X_test1= activations17
X_test1= X_test1['flatten']
X_test2= activations18
X_test2= X_test2['flatten']
X_test3= activations19
X_test3= X_test3['flatten']
X_test4= activations20
X_test4= X_test4['flatten']


In [ ]:
X_test =np.concatenate((X_test1,X_test2,X_test3,X_test4), axis=0)


In [ ]:
import pandas as pd
from sklearn.svm import SVC

from matplotlib import pyplot as plt
%matplotlib inline

In [ ]:
model1 = SVC(kernel='rbf',probability=True, C=10000)

In [ ]:
model1.fit(X_train, Y_train)

In [ ]:
predictions1 = model1.predict(X_train)

In [ ]:
predictions2 = model1.predict(X_test)

In [ ]:
percentage = model1.score(X_test,Y_test)

In [ ]:
from sklearn.metrics import accuracy_score
print("Dataset: train=%d, test=%d" % (X_train.shape[0], Y_test.shape[0]))
score_train = accuracy_score(Y_train, predictions1)
score_test = accuracy_score(Y_test, predictions2)

print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
res = confusion_matrix(Y_test, predictions2)
for i in range(0,len(res)):
  print(res[i])
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(X_test)}")
print(f"Accuracy = {score_test*100} %")
print(classification_report(Y_test, predictions2))

[1496  523]
[ 600 1383]
Confusion Matrix
[[1496  523]
 [ 600 1383]]
Test Set: 4002
Accuracy = 71.93903048475762 %
              precision    recall  f1-score   support

           0       0.71      0.74      0.73      2019
           1       0.73      0.70      0.71      1983

    accuracy                           0.72      4002
   macro avg       0.72      0.72      0.72      4002
weighted avg       0.72      0.72      0.72      4002

